In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
spam = pd.read_csv("spam.csv")
spam

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
spam.shape

(5572, 2)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(spam['Category'])
print(y)

[0 0 1 ... 0 0 0]


In [ ]:
mensagens = spam['Message'].values
X_train, X_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)

In [ ]:
token = Tokenizer(num_words=1000)
token.fit_on_texts(X_train)
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [ ]:
print(X_train)

[[449, 14, 76, 60, 107, 596, 26, 2, 320, 40, 597, 679, 374], [14, 9, 4, 53, 225, 26, 21, 257, 5, 2], [6, 16, 4, 680, 847, 120, 9, 406, 18, 138, 191, 34, 6, 158, 47, 120, 108, 83, 734, 120, 681, 31, 23, 266, 17, 19], [915, 88, 24, 1, 69, 2, 493, 5, 80, 1, 309, 28, 27, 407], [53, 494, 11, 682, 50, 450, 20, 101, 7, 67, 64, 110, 3, 28, 20, 735, 22, 3, 90, 47, 7, 81, 1, 451, 7, 179, 3, 171], [49, 683, 28, 44, 49, 51, 5, 542, 336, 28, 31, 542, 87, 639, 56, 736, 12], [46, 26, 241, 566, 43, 2, 790, 8, 57, 358], [143, 1, 543, 2, 68, 2, 138, 14, 24, 473, 392, 77, 23, 26, 77, 8, 359, 598, 33, 5, 131, 1, 737, 172, 8, 48, 11], [207, 51, 5, 14, 474, 67, 9, 39, 65, 48, 81, 7], [516, 475, 43, 50, 118, 3, 16, 109, 452, 45, 1, 347, 393, 51, 544, 310, 1, 337, 167, 7, 108, 147, 5, 453, 42, 41, 35, 57, 2, 52, 29, 16, 4, 916, 258, 7, 48, 5, 284, 34, 13, 517, 37, 3, 36, 149, 149, 203, 12, 56, 13, 139, 9, 267], [6, 31, 495, 684, 50, 211, 454, 4, 338, 82, 228, 64], [185, 40, 9, 95, 408, 567, 2, 191, 6, 13, 151

In [ ]:
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

In [ ]:
len(token.word_index)

7480

In [ ]:
X_train

array([[449,  14,  76, ...,   0,   0,   0],
       [ 14,   9,   4, ...,   0,   0,   0],
       [  6,  16,   4, ...,   0,   0,   0],
       ...,
       [205,  34,  51, ...,   0,   0,   0],
       [163,   3,   8, ...,   0,   0,   0],
       [ 56,   6,  55, ...,   0,   0,   0]], dtype=int32)

In [ ]:
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=50,input_length=500))
modelo.add(Flatten())
modelo.add(Dense(units=10, activation='relu'))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1, activation='sigmoid'))

In [ ]:
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [ ]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           374000    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense (Dense)               (None, 10)                250010    
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 624,021
Trainable params: 624,021
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelo.fit(X_train,y_train, epochs=30, batch_size=15, verbose=True, validation_data=(X_test, y_test))

Epoch 1/30
260/260 [==============================] - 5s 15ms/step - loss: 0.0814 - accuracy: 0.9115 - val_loss: 0.0438 - val_accuracy: 0.9683
Epoch 2/30
260/260 [==============================] - 5s 18ms/step - loss: 0.0374 - accuracy: 0.9818 - val_loss: 0.0376 - val_accuracy: 0.9773
Epoch 3/30
260/260 [==============================] - 4s 15ms/step - loss: 0.0290 - accuracy: 0.9890 - val_loss: 0.0324 - val_accuracy: 0.9815
Epoch 4/30
260/260 [==============================] - 4s 14ms/step - loss: 0.0229 - accuracy: 0.9923 - val_loss: 0.0296 - val_accuracy: 0.9803
Epoch 5/30
260/260 [==============================] - 4s 17ms/step - loss: 0.0188 - accuracy: 0.9941 - val_loss: 0.0274 - val_accuracy: 0.9803
Epoch 6/30
260/260 [==============================] - 4s 15ms/step - loss: 0.0148 - accuracy: 0.9962 - val_loss: 0.0256 - val_accuracy: 0.9779
Epoch 7/30
260/260 [==============================] - 4s 14ms/step - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.0248 - val_accuracy: 0.9773

In [ ]:
loss, accuracy = modelo.evaluate(X_test,y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

53/53 [==============================] - 1s 9ms/step - loss: 0.0176 - accuracy: 0.9809
Loss: 0.017559941858053207
Accuracy: 0.980861246585846


In [ ]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

53/53 [==============================] - 0s 6ms/step
[[1.0000000e+00]
 [1.7156336e-17]
 [1.9115518e-14]
 ...
 [1.0112711e-14]
 [9.9971431e-01]
 [4.4383286e-12]]


In [ ]:
prev = (nova_previsao > 0.5)
print(prev)

[[ True]
 [False]
 [False]
 ...
 [False]
 [ True]
 [False]]


In [ ]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[1438    6]
 [  26  202]]
